# NBA

## Nos centramos solo en los jugadores importantes ( más de 27 min por partido)

In [2]:
import pandas as pd

df = pd.read_csv('NbaStats.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RANK    0 non-null      float64
 1   NAME    503 non-null    object 
 2   TEAM    503 non-null    object 
 3   POS     503 non-null    object 
 4   AGE     503 non-null    float64
 5   GP      503 non-null    int64  
 6   MPG     503 non-null    object 
 7   USG%    503 non-null    float64
 8   TO%     503 non-null    float64
 9   FTA     503 non-null    int64  
 10  FT%     503 non-null    float64
 11  2PA     503 non-null    int64  
 12  2P%     503 non-null    float64
 13  3PA     503 non-null    int64  
 14  3P%     503 non-null    float64
 15  eFG%    503 non-null    float64
 16  TS%     503 non-null    float64
 17  PPG     503 non-null    float64
 18  RPG     503 non-null    float64
 19  APG     503 non-null    float64
 20  SPG     503 non-null    float64
 21  BPG     503 non-null    float64
 22  TP

Hacemos un info y nos damos cuenta que para filtrar por los jugadores que tengan mas de 27 minutos tenemos que converir los datos de tipo objeto a tipo float, una vez hecho filtramos y mostramos los 5 primeros


In [3]:
df['MPG'] = pd.to_numeric(df['MPG'], errors='coerce')
df['MPG'] = df['MPG'].astype(float)
df = df[df['MPG'] > 27.0]
df.head()

,RANK,NAME,TEAM,POS,AGE,GP,MPG,USG%,TO%,FTA,...,APG,SPG,BPG,TPG,P+R,P+A,P+R+A,VI,ORtg,DRtg
0,NaN,Joel Embiid,Phi,C,29.7,19,34.5,36.7,15.5,220,...,6.4,1.1,1.9,3.9,44.8,39.8,51.3,15.6,126.4,105.2
1,NaN,Luka Doncic,Dal,G,24.8,21,36.3,34.8,14.7,171,...,8.5,1.3,0.6,3.9,40.3,40.5,48.9,14.5,121.2,110.8
2,NaN,Kevin Durant,Pho,F,35.2,19,36.7,32.8,15.1,158,...,5.7,0.7,1.2,3.6,37.5,36.7,43.2,11.4,122.0,111.9
3,NaN,Giannis Antetokounmpo,Mil,F,29.0,22,34.9,33.4,17.1,228,...,5.1,1.4,1.3,3.9,41.3,35.7,46.4,13.6,122.5,107.4
4,NaN,De'Aaron Fox,Sac,G,26.0,16,36.2,33.1,9.3,130,...,6.6,1.4,0.3,2.3,35.0,37.0,41.6,10.8,120.1,111.2


Como el ranking de cada uno no nos interesa, nos vamos a quedar con las columnas que nos interesan realmente



In [4]:
df = df[['NAME','POS','AGE','PPG','RPG']]

una vez tenemos el dataframe con las columnas que necesitaremos vamos a empezar con nuestras hipotesis, no sin antes instalar la libreria que vamos a utilizar


In [5]:
!pip install pingouin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=d010142e5033f2114e133350edd750615caea8e40d86c7d1b54bcd9905feb3c9
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


### - Me da la sensación de que durante este curso los jugadores importantes están metiendo muchísimos puntos, creo que alrededor de unos 18ppp, compruébalo.

H0 - Los jugadores importantes estan metiendo 18 puntos por partido

H1 - Los jugadores importantes estan meiendo mas de 18 puntos por partido

In [13]:
import pingouin as pg
print('La media de puntos es: ', df['PPG'].mean())
test_result = pg.ttest(df['PPG'],18, alternative='greater')
print(test_result)

La media de puntos es:  17.805797101449276
               T  dof alternative     p-val         CI95%   cohen-d   BF10  \
T-test -0.378956  137     greater  0.647346  [16.96, inf]  0.032259  0.203   

           power  
T-test  0.021591  


Como vemos, p es mayor de 0.5 podemos afirmar que la H0 es correcta y la H1 queda descartada como hipotesis.

### - Además me da la sensación de que los jugadores jóvenes sub 27 están dando un paso adelante y meten más puntos que los mayores.

H0 - Los jugadores sub 27 meten los mismos puntos puntos que los mayores

H1 - Los jugadores sub 27 meten mas puntos que los mayores

In [10]:
mean_sub = df[df['AGE'] < 27]['PPG'].mean()
mean_top = df[df['AGE'] > 27]['PPG'].mean()
test_result = pg.ttest(df[df['AGE'] < 27]['PPG'],df[df['AGE'] > 27]['PPG'], alternative='greater')
print(test_result)

               T         dof alternative    p-val         CI95%  cohen-d  \
T-test  0.383608  133.524672     greater  0.35094  [-1.31, inf]  0.06557   

         BF10    power  
T-test  0.392  0.10314  


Como la p es superior a 0.05 la hipotesis nula no se rechaza por lo tanto no podemos afiermar que los sub 27 meten mas puntos que los mayores

### - Me gustaría comprobar  si hay diferencia entre posiciones, y cual son en las posiciones en las que los jugadores meten más puntos.

In [16]:
#Visualizamos la media de puntos por posicion
print(df.groupby('POS')['PPG'].mean())

POS
C      16.590476
F      17.607547
G      18.392063
G-F    16.900000
Name: PPG, dtype: float64


In [20]:
pairwise= pg.pairwise_tests(data=df,dv='PPG',between='POS',alternative='greater')
print(pairwise)

  Contrast  A    B  Paired  Parametric         T         dof alternative  \
0      POS  C    F   False        True -0.649104   32.292385     greater   
1      POS  C    G   False        True -1.125300   34.977921     greater   
2      POS  C  G-F   False        True -0.224471   20.000000     greater   
3      POS  F    G   False        True -0.711574  113.999848     greater   
4      POS  F  G-F   False        True  0.950820   52.000000     greater   
5      POS  G  G-F   False        True  1.834141   62.000000     greater   

      p-unc   BF10    hedges  
0  0.739569  0.626 -0.177121  
1  0.865938  0.877 -0.277894  
2  0.587666  0.466 -0.048984  
3  0.760909  0.498 -0.129780  
4  0.173047  0.459  0.130605  
5  0.035716  1.332  0.231080  


Los G son los que meten mas puntos pero no es una diferencia significativa, solo si los comparas con los G-F, entonces si que podemos afirmas que hay una diferencia signicativa.


### - Me gustaría comprobar si hay diferencia entre posiciones, y cual son en las posiciones en las que los jugadores cogen más rebotes.

In [21]:
#Visualizamos la media de rebotes por posicion
print(df.groupby('POS')['RPG'].mean())

POS
C      9.623810
F      5.894340
G      4.315873
G-F    3.400000
Name: RPG, dtype: float64


In [22]:
pairwise= pg.pairwise_tests(data=df,dv='RPG',between='POS',alternative='greater')
print(pairwise)

  Contrast  A    B  Paired  Parametric          T        dof alternative  \
0      POS  C    F   False        True   6.784367  34.309136     greater   
1      POS  C    G   False        True  10.261949  27.770665     greater   
2      POS  C  G-F   False        True  13.097534  20.000000     greater   
3      POS  F    G   False        True   4.592785  99.449846     greater   
4      POS  F  G-F   False        True   9.025278  52.000000     greater   
5      POS  G  G-F   False        True   4.483069  62.000000     greater   

          p-unc       BF10    hedges  
0  4.015831e-08   5.91e+06  1.792064  
1  2.971682e-11  3.164e+13  2.965760  
2  1.426341e-11  6.054e+08  2.858116  
3  6.415387e-06   3072.844  0.866265  
4  1.586204e-12  5.495e+09  1.239717  
5  1.619873e-05   1213.256  0.564814  


Podemos afirmar que los C tienen una diferencia significativa respecto a los F,G y G-F, que los F tienen una diferencia significativa respecto a los G y G-F y por ultimo que los G tienen una diferencia significativa respecto a los G-F

### - ¿Se están metiendo más puntos de media por posicion que respecto al año anterior?

In [25]:
df_2024 = df
df_2023 = pd.read_csv('NbaStats2023.csv')



df_2023.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RANK    0 non-null      float64
 1   NAME    609 non-null    object 
 2   TEAM    609 non-null    object 
 3   POS     609 non-null    object 
 4   AGE     609 non-null    float64
 5   GP      609 non-null    int64  
 6   MPG     609 non-null    float64
 7   USG%    609 non-null    float64
 8   TO%     608 non-null    float64
 9   FTA     609 non-null    int64  
 10  FT%     609 non-null    float64
 11  2PA     609 non-null    int64  
 12  2P%     609 non-null    float64
 13  3PA     609 non-null    int64  
 14  3P%     609 non-null    float64
 15  eFG%    606 non-null    float64
 16  TS%     606 non-null    float64
 17  PPG     609 non-null    float64
 18  RPG     609 non-null    float64
 19  APG     609 non-null    float64
 20  SPG     609 non-null    float64
 21  BPG     609 non-null    float64
 22  TP

In [26]:
df_2023 = df_2023[df_2023['MPG'] > 27.0]
df_2023.head()

,RANK,NAME,TEAM,POS,AGE,GP,MPG,USG%,TO%,FTA,...,APG,SPG,BPG,TPG,P+R,P+A,P+R+A,VI,ORtg,DRtg
0,NaN,Joel Embiid,Phi,C-F,29.1,66,34.6,37.0,14.5,771,...,4.2,1.0,1.7,3.4,43.2,37.2,47.4,13.0,124.4,104.1
1,NaN,Luka Doncic,Dal,F-G,24.1,66,36.2,37.7,14.0,694,...,8.0,1.4,0.5,3.6,41.0,40.4,49.0,14.4,120.0,109.2
2,NaN,Damian Lillard,Por,G,32.7,58,36.3,33.8,13.7,558,...,7.3,0.9,0.3,3.3,36.9,39.5,44.3,11.5,126.4,117.6
3,NaN,Shai Gilgeous-Alexander,Okc,G-F,24.7,68,35.5,32.8,12.2,739,...,5.5,1.6,1.0,2.8,36.2,36.9,41.7,10.6,124.9,109.8
4,NaN,Giannis Antetokounmpo,Mil,F,28.3,63,32.1,38.8,16.2,772,...,5.7,0.8,0.8,3.9,42.9,36.8,48.6,15.9,116.8,100.6


In [30]:
#Visualizamos la media de rebotes por posicion
print('Media 2023 ppg/posicion ',df_2023.groupby('POS')['PPG'].mean())
print('')
print('Media 2024 ppg/posicion ',df_2024.groupby('POS')['PPG'].mean())

Media 2023 ppg/posicion  POS
C      15.860000
C-F    19.550000
F      17.420455
F-C    18.109091
F-G    20.025000
G      17.707576
G-F    16.717647
Name: PPG, dtype: float64

Media 2024 ppg/posicion  POS
C      16.590476
F      17.607547
G      18.392063
G-F    16.900000
Name: PPG, dtype: float64
